In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
serper_api_key =os.getenv("SERPER_API_KEY")

In [33]:
from crewai import Agent,Task,Crew,LLM


In [34]:

llm = LLM(
    model="ollama/deepseek-r1:1.5b",
    base_url="http://localhost:11434"
)

### Creating Agents 


In [35]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
    "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [36]:
lead_sales_rep_agent =Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with Personalized,Compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    llm=llm,
    verbose=True,
)

### Creating Tools

**CrewAi Tools**

In [37]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [38]:
directory_read_tool = DirectoryReadTool(directory="./instructions")
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

**Custom Tool**

In [39]:
from crewai.tools import  BaseTool

In [40]:
class SentimentAnalysisTool(BaseTool):
    name:str = "Sentiment Analysis Tool"
    description:str= ("Analyzes the sentiment of text "
    "to ensure positive and engaging communication.")

    def _run(self,text:str) ->str:
        # our custom code tool goes here
        return "positive"
        

In [41]:
sentiment_analysis_tool= SentimentAnalysisTool()

### Creating Tasks

In [42]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent
)

In [43]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent
)

### Creating Crew

In [44]:
crew = Crew(
    agents=[sales_rep_agent,lead_sales_rep_agent],
    tasks=[lead_profiling_task,personalized_outreach_task],
    verbose=True,
    memory=True,
    embedder={
        "provider": "ollama",
        "config": {"model": "mxbai-embed-large"}  # Local embedding model
    }
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5c5bc8b3-afe0-4fa1-bdbd-3c4686f235fe                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-08-04 01:13:42,052 - 8647762112 - rag_storage.py-rag_storage:132 - ERROR: Error during short_term search: Expected Embedings to be non-empty list or numpy array, got [] in query.
2025-08-04 01:13:42,055 - 8647762112 - rag_storage.py-rag_storage:132 - ERROR: Error during entities search: Expected Embedings to be non-empty list or numpy array, got [] in query.


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Alright, I need to analyze DeepLearningAI based on the provided information. First, looking at their company   │
│  background, they're part of the CrewAI sales team. Their mission is to provide leads by leveraging data        │
│  analytics and interactions.                                                                                    │
│                                                                                                                 │
│  I'll start with tools available: List files in directory, Read a file's content, Search the internet with      │
│  Serper. The search query tool seems relevant for research on competitors or industry trends. I might look up   │
│  the latest news about DeepLearningAI to see if they've made any recent changes or new focuses.                 │
│                                                                                                                 │
│  For the key personnel section, without specific data, it's challenging, but maybe I can infer based on their   │
│  role as a sales representative and product manager. I should check if any internal reports or team profiles    │
│  are available that detail their current leadership and team structure.                                         │
│                                                                                                                 │
│  Recent business developments would be found by using the search tool with relevant keywords like               │
│  "DeepLearningAI recent updates" or "Company news." This might reveal new initiatives, partnerships, or         │
│  significant achievements.                                                                                      │
│                                                                                                                 │
│  Potential needs from our offer could stem from customer feedback on specific features or services. By          │
│  analyzing their product roadmap and any feedback surveys they've conducted, I can identify gaps in their       │
│  offerings that we can address with tailored solutions.                                                         │
│                                                                                                                 │
│  I should make sure all information is extracted accurately without assumptions, relying solely on the tools    │
│  provided for research and analysis.                                                                            │
│  </think>                                                                                                       │
│                                                                                                                 │
│  Thought: I need to gather details about DeepLearningAI using available tools. The search tool will look up     │
│  relevant keywords from recent updates and news. Tools like List files in directory can help analyze company    │
│  documentation, while Read a file's content may be used to examine internal reports.                            │
│                                                                                                                 │
│  Action: Search the internet with Serper               

2025-08-04 01:13:54,684 - 8647762112 - rag_storage.py-rag_storage:103 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.


...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 111eee7e-26d2-4425-8e5a-37b483e4da35                                                                     │
│  Agent: Sales Representative                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-08-04 01:15:13,108 - 8647762112 - rag_storage.py-rag_storage:132 - ERROR: Error during short_term search: Expected Embedings to be non-empty list or numpy array, got [] in query.
2025-08-04 01:15:13,110 - 8647762112 - rag_storage.py-rag_storage:132 - ERROR: Error during entities search: Expected Embedings to be non-empty list or numpy array, got [] in query.


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Task: Using the insights gathered from the lead profiling report on DeepLearningAI, craft a personalized       │
│  outreach campaign aimed at Andrew Ng, the CEO of DeepLearningAI. The campaign should address their recent      │
│  product launch and how our solutions can support their goals. Your communication must resonate with            │
│  DeepLearningAI's company culture and values, demonstrating a deep understanding of their business and needs.   │
│  Don't make assumptions and only use information you absolutely sure about.                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Alright, let me break this down. I'm DeepLearningAI's lead sales rep, and I need to craft a personalized       │
│  outreach campaign for Andrew Ng, who is CEO of DeepLearningAI. The goal is to address their recent product     │
│  launch and how our solutions can support their goals.                                                          │
│                                                                                                                 │
│  First, I should look into the company's current state using available tools. Since we don't have internal      │
│  data, I'll focus on public information from competitors or industry trends. Tools like Search the internet     │
│  with Serper could be useful here. I might search for news articles, press releases, or any recent updates      │
│  from DeepLearningAI to find new initiatives or focuses.                                                        │
│                                                                                                                 │
│  Next, considering Andrew Ng's role as both a lead representative and a product manager, I should look at his   │
│  recent job responsibilities, such as optimizing sales calls, understanding customer needs, and driving         │
│  revenue growth. This can help in tailoring the campaign specifically for him.                                  │
│                                                                                                                 │
│  I need to research key developments—what has been successful recently, what challenges they're facing, and     │
│  how we can align our solutions with their future goals. The tone should be engaging but professional,          │
│  reflecting DeepLearningAI's corporate identity. Maybe highlight areas where they excel or show opportunities   │
│  for growth.                                                                                                    │
│                                                                                                                 │
│  Potential needs from our offer might stem from customer feedback on specific features or services.             │
│  Identifying these gaps will allow us to provide tailored solutions that address both immediate and future      │
│  challenges.                                                                                                    │
│                                                                                                                 │
│  I also need to ensure all information is accurately extracted using the tools provided without making          │
│  assumptions beyond what's available. Focus on current business trends, recent news, and team insights to       │
│  craft a comprehensive and personalized campaign.                                                               │
│                                                                                                                 │
│  Once I have this data, I'll structure the outreach into drafts tailored for Andrew Ng. Each draft should       │
│  include a compelling narrative connecting our solutions with their recent achievements and future goals,       │
│  ensuring it resonates with their company culture and v

2025-08-04 01:15:26,924 - 8647762112 - rag_storage.py-rag_storage:103 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.


In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

<think>
Alright, let me break this down. The user wants a detailed analysis of DeepLearningAI using specific tools like sentiment analysis and search. First, I need to figure out how to simulate gathering info without real data. Maybe I'll use the search tool to look up recent articles or press releases about DeepLearningAI.

Next, for key personnel, I can create JSON objects of each role and use the search tool to find relevant information. For example, checking if any recent news mentions the CEO's comments or a new product launch. That should give me some key points on company background.

For business developments, I'll query specific keywords like "new AI breakthroughs" or "global expansion" in the search tool. This will help identify where DeepLearningAI is moving and what new initiatives they've taken.

As for potential needs, researching industry trends with the same tools can show areas where competitors are innovating, which I can then compare to DeepLearningAI's current offerings. This analysis should highlight any gaps or opportunities that could benefit from DeepLearningAI's solutions.

I also notice that without real-time data access, my analysis will have placeholders and explanations for missing information. So, the final report will include specific details where possible but note areas that need further research.

Overall, by systematically using these tools and structuring the report with clear sections, I can provide a comprehensive analysis as requested.
</think>

**Final Answer**
```
Thought: I now know the complete analysis of DeepLearningAI based on the search query "Recent AI Innovations in the Market" and used all available tools including Sentiment Analysis Tool, Search the internet with Serper.
Action: Search the internet with Serper